In [33]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from matplotlib.colors import TwoSlopeNorm

import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper, tfr_morlet
from mne.time_frequency import morlet, fwhm
import tables as tb


## Save in PyTables 

In [5]:

# Define the structure for the table
class SubjectData(tb.IsDescription):
    P_id = tb.StringCol(16)  # Participant ID
    Averaged_Epochs = tb.Float64Col(shape=(4,))  # Averaged epochs
    ICA_Components = tb.Float64Col(shape=(2,))  # ICA components
    Alpha_Left_ERD = tb.Float64Col()  # Alpha left ERD
    Alpha_Right_ERD = tb.Float64Col()  # Alpha right ERD
    Beta_Left_ERD = tb.Float64Col()  # Beta left ERD
    Beta_Right_ERD = tb.Float64Col()  # Beta right ERD

# Create a new HDF5 file
hdf5_file = tb.open_file("subject_data.h5", mode="w", title="Subject Data")

# Create a group for subjects
subject_group = hdf5_file.create_group("/", "Subjects", "Subject Data")

# Create a table for each subject
for subject_id in range(1, 51):
    table = hdf5_file.create_table(subject_group, f"Subject_{subject_id}", SubjectData, f"Subject {subject_id} Data")

# Close the HDF5 file
hdf5_file.close()



## Preprocess

In [21]:


%matplotlib qt

def read_file(path):
    r = mne.io.read_raw_bdf(path, preload=False)
    
    # Define signal info
    n_time_samps = r.n_times
    time_secs = r.times
    ch_names = r.ch_names
    n_chan = len(ch_names) 
    
    
    channels_to_keep = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10",
            "A11", "A12", "A13", "A14", "A15", "A16", "A17", "A18", "A19", "A20",
            "A21", "A22", "A23", "A24", "A25", "A26", "A27", "A28", "A29", "A30",
            "A31", "A32", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10",
            "B11", "B12", "B13", "B14", "B15", "B16", "B17", "B18", "B19", "B20",
            "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B29", "B30", "B31", "B32"]

    channels_to_exclude_str = [ch for ch in r.ch_names if ch not in channels_to_keep + ['Status']]
    
    
    raw = mne.io.read_raw_bdf(path, exclude = channels_to_exclude_str ,  preload=True)
    
    list_chan_name = { 'A3': 'AF3',
     'A7': 'F7', 'A6': 'F5','A5': 'F3', 'A4': 'F1',
     'A8': 'FT7', 'A9': 'FC5','A10': 'FC3', 'A11': 'FC1',
     'A15': 'T7', 'A14': 'C5','A13': 'C3', 'A12': 'C1',
     'A16': 'TP7', 'A17': 'CP5','A18': 'CP3', 'A19': 'CP1', 'A32': 'CPz',
     'A24': 'P9', 'A23': 'P7','A22': 'P5', 'A21': 'P3', 'A20': 'P1','A31': 'Pz',
     'A25': 'PO7', 'A26': 'PO3','A30': 'POz', 
     'A27': 'O1',   'A29': 'Oz', 
     'A28': 'Iz', 
     'B1': 'Fpz','B2': 'Fp2',
     'B5': 'AFz', 'B4': 'AF4','B3': 'AF8',
     'B6': 'Fz', 'B7': 'F2','B8': 'F4', 'B9': 'F6','B10': 'F8',
     'B15': 'FCz', 'B14': 'FC2','B13': 'FC4', 'B12': 'FC6','B11': 'FT8',
     'B16': 'Cz', 'B17': 'C2','B18': 'C4', 'B19': 'C6','B20': 'T8',
     'B24': 'CP2', 'B23': 'CP4','B22': 'CP6', 'B21': 'TP8',
     'B25': 'P2', 'B26': 'P4','B27': 'P6', 'B28': 'P8','B29': 'P10',
     'B31': 'PO4', 'B30': 'PO8',
     'B32': 'O2'
     }

    _ = raw.rename_channels(list_chan_name)
    print(raw.info)
    
    return raw

    
def highlow_pass(raw):
    # We cut off at one Hertz, because we will be looking at the alpha and beta frequency bands that start
# from the 8-10 Hz. Apply high-pass filter
    raw.filter(l_freq=1, h_freq=None, fir_design='firwin', filter_length='auto',
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
           phase='zero', fir_window='hamming', verbose=True)

# Apply low-pass filter if needed
    raw.filter(l_freq=None, h_freq=40, fir_design='firwin', filter_length='auto',
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', method='fir',
           phase='zero', fir_window='hamming', verbose=True)

    return raw 
    
    
def plotsignal(signal, channels, pick_min, pick_max):
    
    ch_names = signal.ch_names
    
    
    picks = ch_names[pick_min:pick_max]  # Pick the first 20 EEG channels
    signal.plot(n_channels=channels, picks=picks, events = False, title='EEG Data for the Channels')
    
    
   
def rename_events(raw):
    
    # Find events and drop unnecessary
    events_ID = mne.find_events(raw, stim_channel="Status")
    events = mne.pick_events(events_ID, exclude=[3, 5, 40, 42, 62])
    
    
    unsuccessful_event_index = None  # For sanity check
    
    for i, event in enumerate(events):
    
        if event[2] == 10:
            
            next_event_index = i + 1    
            while next_event_index < len(events) and events[next_event_index][2] == 61: 
                events[next_event_index][2] = 610 # Modify the trigger value
                next_event_index += 1   
        
        if event[2] == 20:
            next_event_index = i + 1    
            while next_event_index < len(events) and events[next_event_index][2] == 61: 
                events[next_event_index][2] = 620 # Modify the trigger value
                next_event_index += 1   
        
        
        if event[2] == 30:
            next_event_index = i + 1    
            while next_event_index < len(events) and events[next_event_index][2] == 61: 
                events[next_event_index][2] = 630 # Modify the trigger value
                next_event_index += 1   

        else:
            unsuccessful_event_index = i

    # Check if any events with trigger value 61 are still present
    if any(event[2] == 61 for event in events):
        print("Renaming was unsuccessful :(")
        if unsuccessful_event_index is not None:
            print("Problematic event index:", unsuccessful_event_index)
            print("Problematic event details:", events[unsuccessful_event_index])
   
    # Define epoch parameters
    tmin = -2
    tmax = 3

    
    
    # Rename events with numerical IDs to have corresponding event names
    event_dict = {"10": 10, 
                  "20": 20,
                  "30": 30,
                  "selftap": 41, 
                  "VP610": 610,
                  "VP620": 620, 
                  "VP630": 630}
    
    # Epoch signal
    epochs = mne.Epochs(raw, events, event_id = event_dict, tmin=tmin, tmax=tmax, baseline=None)
    
 

    # Might introduce edge artifacts for each epoch.
    epochs.load_data()
    epochs_rs = epochs.resample(sfreq=256)

    # Print events
    event_ids = epochs_rs.events[:, -1]
    print()
    unique_event_ids = set(event_ids)
    print("Unique Event IDs:", unique_event_ids)
    # Print counts of each event ID
    for event_id in unique_event_ids:
        count = (event_ids == event_id).sum()
        print("Event ID:", event_id, "Count:", count)

    return epochs

def drop_bad_interp(epochs, channels_list):
    
    ep_ds = epochs.copy()

    # Highlight bad channels # we keep the flat one as it will be fixed after averaging 
    _ = ep_ds.info["bads"].extend(channels_list)  

    # Set montage and interpolate bads 
    ep_ds.set_montage('standard_1020')
    ep_interp = ep_ds.interpolate_bads(
    reset_bads=False, method='MNE', verbose=True
    )
    
    return ep_interp
    
    


def delete_variables_except(exceptions):
    """
    Delete all variables in the global namespace except for the specified exceptions.
    
    Parameters:
    exceptions (list): List of variable names to exclude from deletion.
    """
    # Get a list of all variable names in the global namespace
    all_variables = list(globals().keys())
    
    # Iterate through all variables and delete those not in the exceptions list
    for var_name in all_variables:
        if var_name not in exceptions:
            # Check if the object is a variable (not a module, function, etc.)
            if not isinstance(globals()[var_name], (types.FunctionType, types.ModuleType)):
                del globals()[var_name]
    
   


## Saving files 

In [22]:
def save_epochs(sub,data_dir,epochs,name):
    # saves Epoch
    epochs.save(data_dir + sub + name + '-epo.fif', overwrite=True)
    
    
def save_ica(subject,directionary,ica):
        
        ica.save(directionary + subject+ '-ica.fif', 
        overwrite=True)

## ICA and waveanalysis

In [23]:
def fit_ica(epochs, sub):
    
    # run fast ICA 
    ica = mne.preprocessing.ICA(
    n_components=30, method="fastica", max_iter="auto", random_state=97)
    
    ica.fit(epochs) # fit on epochs
    
    # Visualise ICA components
    montage = mne.channels.make_standard_montage("standard_1005") 
    epochs.set_montage(montage)

    ica.plot_sources(epochs) # plot the raw signal 

    #scalp field topographies 
    ica.plot_components() # plot a Topomap and Chooose ICA compoenents.
    data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/Ica_files/'
    
    

    save_ica(sub,data_dir,ica)
    
    return ica
    

def get_ica_sources(epochs, list_of_Ica_components, ica):
    
    
    all_sources = ica.get_sources(epochs)

    
    # Get the sources corresponding to the components
    motor_source = all_sources.pick(list_of_Ica_components)
    
    # separate conditions
    motor_source_event41 = motor_source['selftap']
    motor_source_event610 = motor_source['VP610']
    motor_source_event620 = motor_source['VP620']
    motor_source_event630 = motor_source['VP630']
    
    
    # combine all conditions to one list 
    all_motor_components = [motor_source_event41, motor_source_event610, motor_source_event620, motor_source_event630]
    
    return all_motor_components




def basecorrection_convert_to_df(p):
    
    # Define baseline period
    baseline_tmin = -0.5
    baseline_tmax = 1.5

    time_interval = [baseline_tmin, baseline_tmax]
    
    interval_baseline = [i for i in p.times if i >= time_interval[0] and i <= time_interval[1]]
    baseline = []
    
    # Find the indices corresponding to the time interval
    start_index = np.where(p.times >= time_interval[0])[0][0]
    end_index = np.where(p.times <= time_interval[1])[0][-1]

    
    # Calculate baseline
    for e in range(len(p.data)):
        avg_epoch = np.mean(p.data[e, :, :, start_index:end_index], axis=-1)
        baseline.append(avg_epoch)
    baseline = np.asarray(baseline)

    
    print("Shape of p.data:", p.data.shape)
    print("Shape of baseline:", baseline.shape)
    
    baseline_reshaped = np.expand_dims(baseline, axis=-1) 
    
    
    
    # Subtract baseline and normalize
    for e in range(len(p.data)):
        for i in range(len(p.ch_names)):
            p.data[e, i, :] -= baseline_reshaped[e, i]  # Subtract baseline
            p.data[e, i, :] /= baseline_reshaped[e, i]  # Normalize

    
    return p
      

def wave_analysis(motor_source):


    # Perform wavelet analysis
    freqs = np.arange(1, 31) # interested in frequencies 1 to 30 hz
    n_cycles = 3 + 0.125 * freqs # Number of cycles   
                                
    p = mne.time_frequency.tfr_morlet(motor_source, freqs, n_cycles, use_fft=False, return_itc=False,

                        decim=1, n_jobs=None, zero_mean=True, average=False, output='power',

                        picks="all")
         
         
    # Output p datandarray, shape (n_epochs, n_channels, n_freqs, n_times)
    
    return  p



      

## Get ERD's

In [24]:
def Get_alpha_beta_comps(Basecorrect_epochs):
    
    
    # Define frequency bands
    alpha_band = [8, 12]
    beta_band = [13, 25]

    # Define time window
    tmin = -0.5
    tmax = 1.5
    baseline = (tmin,tmax)

    
    cropped_epochs = Basecorrect_epochs.copy().crop(tmin, tmax)
    #cropped_epochs  = Basecorrect_epochs.apply_baseline(baseline)

    
    p_alpha_left = cropped_epochs.copy().crop(fmin = alpha_band[0], fmax = alpha_band[1]).pick(picks = ['ICA009']).average(dim = 'freqs')
    p_beta_left = cropped_epochs.copy().crop(fmin = beta_band[0], fmax = beta_band[1]).pick(picks = ['ICA009']).average(dim = 'freqs')
    p_alpha_right = cropped_epochs.copy().crop(fmin = alpha_band[0], fmax = alpha_band[1]).pick(picks = ['ICA008']).average(dim = 'freqs')
    p_beta_right = cropped_epochs.copy().crop(fmin = beta_band[0], fmax = beta_band[1]).pick(picks = ['ICA008']).average(dim = 'freqs')

    
    
    print("Shape of epochs data averaged across alpha band for left component:", p_alpha_left.data.shape)
    print("Timepoints for above mentioned data:", len(p_alpha_left.times))
    
    # Compute ERD &  # datandarray, shape (n_channels, n_freqs, n_times)
    erd_alpha_left = p_alpha_left.average(dim = 'epochs')
    erd_beta_left = p_beta_left.average(dim = 'epochs')
    erd_alpha_right = p_alpha_right.average(dim = 'epochs')
    erd_beta_right = p_beta_right.average(dim = 'epochs')
    
    print(" ")
    print("Shape of epochs data averaged across trials for left component:", erd_alpha_left.data.shape)
    
    return erd_alpha_left, erd_alpha_right, erd_beta_left, erd_beta_right
    #return p_alpha_left, p_alpha_right, p_beta_left, p_beta_right
    


In [25]:
# Loads data, filters raw signal, artefact rejection, average reference
sample_fname = "C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/subj_07_070223_task.bdf"
raw_filt = highlow_pass(read_file(sample_fname))  # reads file and performs high and low pass filtering
epochs_rs = rename_events(raw_filt)# renames events, create epochs, resample to 256HZ



Extracting EDF parameters from C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\subj_07_070223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\subj_07_070223_task.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4517887  =      0.000 ...  2206.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: A1, A2, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, ...
 chs: 64 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 417.0 Hz
 meas_date: 2023-02-07 14:46:06 UTC
 nchan: 65
 projs: []
 sfreq: 2048.0 Hz
 subject_info: 1 item (dict)
>
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   15.1s finished


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 677 samples (0.331 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   12.8s finished


Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
4237 events found on stim channel Status
Event IDs: [ 3  5 10 20 30 40 41 42 61 62]
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1707 events and 10241 original time points ...
0 bad epochs dropped

Unique Event IDs: {610, 41, 10, 620, 20, 630, 30}
Event ID: 610 Count: 450
Event ID: 41 Count: 540
Event ID: 10 Count: 30
Event ID: 620 Count: 301
Event ID: 20 Count: 30
Event ID: 630 Count: 326
Event ID: 30 Count: 30


In [26]:
ep_interp = drop_bad_interp(epochs_rs,['TP8']) # artefact rejection and interpolation of channels

# use the average of all channels as reference
ep_avg_ref = ep_interp.set_eeg_reference(ref_channels="average")

# Save epochs in events.
epochs_trial41 = ep_avg_ref['selftap']
epochs_trial610 = ep_avg_ref['VP610']
epochs_trial620 = ep_avg_ref['VP620']
epochs_trial630 = ep_avg_ref['VP630']


p_id = 'sub-007'
data_dir = 'C:/Users/Mabel Ife/OneDrive - Danmarks Tekniske Universitet/Dokumenter/Master in AI/Final Thesis/Data/epochs/'

save_epochs(p_id,data_dir,epochs_trial41, '-AVtrial41')
save_epochs(p_id,data_dir,epochs_trial610, '-AVtrial610')
save_epochs(p_id,data_dir,epochs_trial620, '-AVtrial620')
save_epochs(p_id,data_dir,epochs_trial630, '-AVtrial630')


Setting channel interpolation method to {'eeg': 'MNE'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.6 mm
    Computing dot products for 63 EEG channels...
    Computing cross products for 63 → 64 EEG channels...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


    Preparing the mapping matrix...
    Truncating at 63/63 components and regularizing with α=1.0e-01
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.


For the subject 7, we found the two components 8 and 9. 

## ICA and Wavelet analysis 

In [27]:
# Free-up memory
delete_variables_except(['ep_avg_ref'])

In [30]:
# fit ICA and look for motor components
p_id = 'sub-007'
ICA = fit_ica(ep_avg_ref,p_id)

# Pick components (comment out when needed)
# ICA.plot_properties(ep_avg_ref, picks=[8,9], log_scale=False)


# get sources for components
all_conditions = get_ica_sources(ep_avg_ref,['ICA008','ICA009'],ICA)

# Run wavelet and basecorrection 

baseCorr_power = [] # saves baselinecorrected epochs 

for motor_source_condition in all_conditions: # loop through each condition
    p_df = basecorrection_convert_to_df(wave_analysis(motor_source_condition))
    
    baseCorr_power.append(p_df)



Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 265.8s.
Not setting metadata
1707 matching events found
No baseline correction applied
0 projection items activated
Using matplotlib as 2D backend.
Writing ICA solution to C:\Users\Mabel Ife\OneDrive - Danmarks Tekniske Universitet\Dokumenter\Master in AI\Final Thesis\Data\Ica_files\sub-007-ica.fif...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.4s remaining:    0.0s


Not setting metadata


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.8s finished


Shape of p.data: (540, 2, 30, 1280)
Shape of baseline: (540, 2, 30)


...

Shape of p.data: (301, 2, 30, 1280)
Shape of baseline: (301, 2, 30)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


Not setting metadata


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.2s finished


Shape of p.data: (326, 2, 30, 1280)
Shape of baseline: (326, 2, 30)


In [31]:
# Get ERD's in alfa and beta frenquencies for each component
erd_alpha_L41, erd_alpha_R41, erd_beta_L41, erd_beta_R41 = Get_alpha_beta_comps(baseCorr_power[0])
erd_alpha_L610, erd_alpha_610, erd_beta_610, erd_beta_610 = Get_alpha_beta_comps(baseCorr_power[1])
erd_alpha_L620, erd_alpha_620, erd_beta_620, erd_beta_620 = Get_alpha_beta_comps(baseCorr_power[2])
erd_alpha_L630, erd_alpha_630, erd_beta_630, erd_beta_630 = Get_alpha_beta_comps(baseCorr_power[3])

Shape of epochs data averaged across alpha band for left component: (540, 1, 1, 513)
Timepoints for above mentioned data: 513
 
Shape of epochs data averaged across trials for left component: (1, 1, 513)
Shape of epochs data averaged across alpha band for left component: (450, 1, 1, 513)
Timepoints for above mentioned data: 513
 
Shape of epochs data averaged across trials for left component: (1, 1, 513)
Shape of epochs data averaged across alpha band for left component: (301, 1, 1, 513)
Timepoints for above mentioned data: 513
 
Shape of epochs data averaged across trials for left component: (1, 1, 513)
Shape of epochs data averaged across alpha band for left component: (326, 1, 1, 513)
Timepoints for above mentioned data: 513
 
Shape of epochs data averaged across trials for left component: (1, 1, 513)


## plot ERD's 


In [34]:
fig, ax = plt.subplots(figsize=(20, 10))

# A
time_points = np.linspace(-0.5, 512 / 256, 513)  # 512 samples with 256 Hz sampling rate


ax.set_title('Plot1:ICA_left, alpha')
ax.set_xlabel('Time Instances')
ax.set_ylabel('Volt')

ax.plot(time_points, erd_alpha_L41.data[0,0,:], color='blue', label='self Tap')
ax.plot(time_points, erd_alpha_L610.data[0,0,:], color='red', label='Non-adaptive')
ax.plot(time_points, erd_alpha_L620.data[0,0,:], color='green', label='Moderately adaptive')
ax.plot(time_points, erd_alpha_L630.data[0,0,:], color='pink', label='Overly adaptive')

# Adding vertical lines at specified time points
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)  # at time point 0
ax.axvline(x=1, color='black', linestyle='--', linewidth=1)  # at time point 1
ax.axvline(x=1.5, color='black', linestyle='--', linewidth=1)  # at time point 1.5

legend = ax.legend(loc='upper right', shadow=True, fontsize='medium')
plt.title('ERDs of different conditions for left ICA component in alpha freq-range')
plt.show()

In [ ]:
def get_alpha_beta_comps(basecorrect_epochs):
    
    
    # Define frequency bands
    alpha_band = [8, 12]
    beta_band = [13, 25]

    # Define time window
    tmin = -0.5
    tmax = 1.5

    # Select components 
    left_component = 1
    right_component = 0

    # Extract relevant data, apply filters, and select components
    p_alpha_left = basecorrect_epochs.copy().crop(tmin, tmax,alpha_band[0], alpha_band[1]).pick(picks = ['ICA009']).average()
    p_beta_left = basecorrect_epochs.copy().crop(tmin, tmax,alpha_band[0], alpha_band[1]).pick(picks = ['ICA009']).average()
    p_alpha_right = basecorrect_epochs.copy().crop(tmin, tmax,alpha_band[0], alpha_band[1]).pick(picks = ['ICA008']).average()
    p_beta_right = basecorrect_epochs.copy().crop(tmin, tmax,alpha_band[0], alpha_band[1]).pick(picks = ['ICA008']).average()

    # Compute ERD
    erd_alpha_left = np.mean(p_alpha_left.data, axis=0)
    erd_beta_left = np.mean(p_beta_left.data, axis=0)
    erd_alpha_right = np.mean(p_alpha_right.data, axis=0)
    erd_beta_right = np.mean(p_beta_right.data, axis=0)
    
    
    return erd_alpha_left, erd_alpha_right, erd_beta_left, erd_beta_right
    
    


    